In [6]:
import pandas as pd
import pickle
from helper_functions import Helper
from helper_functions import CoinAPI
import requests

In [7]:
if __name__ == "__main__":
    # import api key
    API_KEY_FP = '../../admin/coinapi.txt'
    with open(API_KEY_FP) as f:
        API_KEY = f.readlines()[0].strip()

    # Set args
    CW_IN_FP = '../data/derived/cm_to_coinapi_cw.pkl'
    ASSET_IN_FP = '../data/clean/asset_universe_dict.pickle'
    BASE_URL   = 'https://rest.coinapi.io/v1/'
    BASE_HEADERS = {'X-CoinAPI-Key': API_KEY}
    LEGIT_US_EXCHANGES = ['BINANCEUS', 'BITSTAMP', 'COINBASE', 'CRYPTOCOM', 'FTXUS', 
        'GEMINI', 'KRAKEN', 'KUCOIN']
    TARGET_FREQ = '1HRS'
    TIME_START = '2016-07-01'
    TIME_END = '2023-01-02'
    PANEL_OUT_FP = '../data/raw/coinapi_panel_hourly.pkl'

    # confirm api is working
    url = 'https://www.coinapi.io/api/subscriptions/usage/rest/history'
    print(requests.get(url, headers=BASE_HEADERS).json())    
    
    # Import asset universe and cw
    cw_df = pd.read_pickle(CW_IN_FP)
    with open(ASSET_IN_FP, "rb") as f:
        asset_universe_dict = pickle.load(f)
    asset_universe_cm = Helper.findUniqueAssets(asset_universe_dict)
    asset_universe = list(cw_df[cw_df.asset_cm.isin(asset_universe_cm)].asset_coinapi.values)

    # pull relevant markets
    markets_df = CoinAPI.pullMarketInfo(BASE_URL, BASE_HEADERS, LEGIT_US_EXCHANGES, asset_universe)

    # pull coinapi aggregated prices
    prices_df = CoinAPI.pullExchangeRates(BASE_URL, BASE_HEADERS, ['USDC', 'USDT'] + asset_universe,
                    TARGET_FREQ, TIME_START, TIME_END)
    macro_df, prices_df = CoinAPI.seperateMacroAndAssetRefPrices(prices_df)

[{'date': '2023-03-21', 'requests': 10944, 'apicalls': 2020}, {'date': '2023-03-20', 'requests': 90483, 'apicalls': 2128}, {'date': '2023-03-19', 'requests': 16728, 'apicalls': 390}, {'date': '2023-03-18', 'requests': 12607, 'apicalls': 226}, {'date': '2023-03-17', 'requests': 11800, 'apicalls': 249}, {'date': '2023-03-15', 'requests': 85274, 'apicalls': 2851}, {'date': '2023-03-14', 'requests': 87973, 'apicalls': 2943}, {'date': '2023-03-08', 'requests': 101011, 'apicalls': 3378}, {'date': '2023-03-07', 'requests': 35546, 'apicalls': 1208}, {'date': '2023-03-06', 'requests': 65646, 'apicalls': 2194}]
Processing asset #1 (0.36%): USDC
Processing asset #2 (0.72%): USDT
Processing asset #3 (1.08%): 1INCH
Processing asset #4 (1.44%): AAVE
Processing asset #5 (1.80%): ACA
Processing asset #6 (2.16%): ACH
Processing asset #7 (2.52%): ADA
Processing asset #8 (2.88%): ADX
Processing asset #9 (3.24%): AERGO
Processing asset #10 (3.60%): AGLD
Processing asset #11 (3.96%): AKRO
Processing asset 

The API call failed with error: ("Connection broken: InvalidChunkLength(got length b'', 0 bytes read)", InvalidChunkLength(got length b'', 0 bytes read))
Retrying after 1.17 seconds.


Processing asset #63 (22.66%): CREAM
Processing asset #64 (23.02%): CRO
Processing asset #65 (23.38%): CRPT
Processing asset #66 (23.74%): CRV
Processing asset #67 (24.10%): CTC
Processing asset #68 (24.46%): CTSI
Processing asset #69 (24.82%): CUDOS
Processing asset #70 (25.18%): CVC
Processing asset #71 (25.54%): CVX
Processing asset #72 (25.90%): DAR
Processing asset #73 (26.26%): DASH
Processing asset #74 (26.62%): DGB
Processing asset #75 (26.98%): DIA
Processing asset #76 (27.34%): DNT
Processing asset #77 (27.70%): DODO
Processing asset #78 (28.06%): DOGE
Processing asset #79 (28.42%): DOT
Processing asset #80 (28.78%): DUSK
Processing asset #81 (29.14%): DYDX
Processing asset #82 (29.50%): EGLD
Processing asset #83 (29.86%): ENJ
Processing asset #84 (30.22%): ENS
Processing asset #85 (30.58%): EOS
Processing asset #86 (30.94%): ETC
Processing asset #87 (31.29%): ETH
Processing asset #88 (31.65%): EWT
Processing asset #89 (32.01%): FARM
Processing asset #90 (32.37%): FET
Process

In [8]:
# TODO TEMP SAVE OF MACRO AND PRICES
markets_df.to_pickle('temp_markets.pkl')
macro_df.to_pickle('temp_macro.pkl')
prices_df.to_pickle('temp_macro.pkl')

In [9]:
# pull market prices and volumes
master_markets_list = list(markets_df.symbol_id.values)

In [ ]:
# TODO  FRIDAY NOON
end = 200
markets_list = master_markets_list[:end] # TODO CHANGE TO SOME LIST
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)

In [ ]:
master_raw_panel_df = pd.DataFrame()
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_200.pkl')

In [ ]:
# TODO SAT
start = 200
end = 400
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_400.pkl')

In [ ]:
# TODO SUN
start = 400
end = 600
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_600.pkl')

In [ ]:
# TODO MON
start = 600
end = 800
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_800.pkl')

In [ ]:
# TODO TUES
start = 800
end = 1000
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_1000.pkl')

In [ ]:
# TODO  W
start = 1000
end = 1200
markets_list = master_markets_list[start:end] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_1200.pkl')

In [ ]:
# TODO Th
start = 1200
markets_list = master_markets_list[start:] 
raw_panel_df = CoinAPI.pullMarketData(BASE_URL, BASE_HEADERS, markets_list, 
                                TARGET_FREQ, TIME_START, TIME_END)
master_raw_panel_df = pd.concat((master_raw_panel_df, raw_panel_df))
master_raw_panel_df.to_pickle('temp_panel_thru_all.pkl')

In [ ]:


# clean the panel
panel_df = CoinAPI.cleanPanel(raw_panel_df, prices_df, macro_df, TARGET_FREQ)
panel_df.to_pickle(PANEL_OUT_FP)